In [ ]:
#######softmax

import tensorflow as tf
import numpy as np 
from tensorflow.examples.tutorials.mnist import input_data


######prepare data
mnist = input_data.read_data_sets('MNIST_data/',one_hot = True);


#######create the graph

x = tf.placeholder(tf.float32,shape = [None,784],name = 'x');
y_ = tf.placeholder(tf.float32,shape = [None,10],name = 'y_');

#initialize weights and bias;
with tf.name_scope('input_weight'):
    W = tf.Variable(tf.zeros([784,10]),name = 'input_weight');
    
with tf.name_scope('input_bias'):
    b = tf.Variable(tf.zeros([10]),name = 'input_bias');

#Predict class and loss function
with tf.name_scope('y'):
    y = tf.nn.softmax(tf.matmul(x,W) + b)
#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_));
#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = y_))
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
tf.summary.scalar('loss_function', cross_entropy)

#train_step
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy);
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)


#####optimization:梯度下降
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)



#Cannot evaluate tensor using `eval()`: No default session is registered. 
#Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

#set sess as default
sess = tf.InteractiveSession();





########create session
#sess = tf.Session();
#init op 
init = tf.global_variables_initializer();
sess.run(init);
####TensorBoard
merged_summary_op = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter('/tmp/mnist_logs',sess.graph)


#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
for i in range(1000):
    batch = mnist.train.next_batch(100);
    sess.run(train_step,feed_dict={x: batch[0], y_: batch[1]});
    summary_str = sess.run(merged_summary_op,feed_dict={x: batch[0], y_: batch[1]});
    summary_writer.add_summary(summary_str, i);
    
    if i % 50 == 0:
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float32"))
        print "Setp: ", i, "Accuracy: ",sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
        




summary_writer.flush()
summary_writer.close()
########evaluate
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(session=sess,feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

In [4]:
import tensorflow as tf  
  
#our NN's output  
logits=tf.constant([[1.0,2.0,3.0],[1.0,2.0,3.0],[1.0,2.0,3.0]])  
#step1:do softmax  
y=tf.nn.softmax(logits)  
#true label  
y_=tf.constant([[0.0,0.0,1.0],[0.0,0.0,1.0],[0.0,0.0,1.0]])
y_value = tf.constant([[2],[2],[2]])
#step2:do cross_entropy  
cross_entropy = -tf.reduce_sum(y_*tf.log(y))  
#do cross_entropy just one step  
cross_entropy2=tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y_))#dont forget tf.reduce_sum()!!  
  
with tf.Session() as sess:  
    softmax=sess.run(y)  
    c_e = sess.run(cross_entropy)  
    c_e2 = sess.run(cross_entropy2)  
    print("step1:softmax result=")  
    print(softmax)  
    print("step2:cross_entropy result=")  
    print(c_e)  
    print("Function(softmax_cross_entropy_with_logits) result=")  
    print(c_e2) 

step1:softmax result=
[[ 0.09003057  0.24472848  0.66524094]
 [ 0.09003057  0.24472848  0.66524094]
 [ 0.09003057  0.24472848  0.66524094]]
step2:cross_entropy result=
1.22282
Function(softmax_cross_entropy_with_logits) result=
1.22282


In [ ]:
#####cnn解决mnist手写体识别.

import tensorflow as tf
import numpy as np 
from tensorflow.examples.tutorials.mnist import input_data

##initial weights and bias
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1);
    return tf.Variable(initial);

def bias_variable(shape):
    initial = tf.constant(0.1,shape = shape);
    return tf.Variable(initial);

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1, 1, 1, 1], padding='SAME');

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

######prepare data
mnist = input_data.read_data_sets('MNIST_data/',one_hot = True);


#######create the graph
x = tf.placeholder(tf.float32,shape = [None,784]);
y_ = tf.placeholder(tf.float32,shape = [None,10]);



##initial weight and bias;
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])


x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


sess = tf.InteractiveSession()

cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

In [ ]:
import tensorflow as tf  
#case 2  
input = tf.Variable(tf.ones([1,3,3,1]))  
filter = tf.Variable(tf.ones([2,2,1,1]))  
  
op2 = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='SAME')  
init = tf.global_variables_initializer()  
with tf.Session() as sess:  
    sess.run(init) 
    
    print filter.eval()
    print input.eval()
    
    print("case 2") 
    
    print(sess.run(op2))

In [1]:
import tensorflow as tf


##build graph;
weights = tf.Variable(tf.ones(10,3),tf.float32);
bias = tf.Variable(tf.ones(3),tf.float32);
x = tf.Variable(tf.ones(5,10),tf.float32)


y_ = tf.matmul(x,weights)+bias;

sess = tf.Session()
init = tf.global_variables_initializer();
sess.run(init);

print sess.run(y_)

TypeError: Value passed to parameter 'a' has DataType bool not in list of allowed values: float16, float32, float64, int32, complex64, complex128